# Image Classification Pipeline using Remo

In this tutorial, Remo will be used to accelerate the process of building a transfer learning pipeline for the task of Image Classification.

In [3]:
#Imports
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.models as models
import pandas as pd
import os
import tqdm
import torch.optim as optim
import random
#import remo
remo.set_viewer('jupyter')
from pprint import pprint
import json
import glob
import random
random.seed(4)

## Adding Data to Remo

- The dataset used in this example is the <a href="http://www.robots.ox.ac.uk/~vgg/data/flowers/102/">Flowers 102 Dataset</a>.
- Run the next cell to download the data from s3, create a new folder and extract the files required.

- The directory structure of the dataset is:

    ```
    ├── small_flowers
        ├── images
            ├── 0
                ├── image_1.jpg
                ├── image_2.jpg
                ├── ...
            ├── 1
                ├── image_3.jpg
                ├── image_4.jpg
                ├── ...
        ├── annotations
            ├── mapping.json
    ```

In [4]:
# The dataset will be downloaded from s3, and extracted into a new folder
!wget https://s-3.s3-eu-west-1.amazonaws.com/small_flowers.zip
!unzip small_flowers.zip

In [5]:
# The path to the folders
path_to_images = "./small_flowers/images"
path_to_annotations = "./small_flowers/annotations"

**Generating Annotations from Folders**

To generate an annotations file from a folder of folders, the path to the root directory containing the class folders is passed to ```remo.generate_annotations_from_folders()```.

In [6]:
remo.generate_annotations_from_folders(path_to_images)

'./small_flowers/images/annotations.csv'

**Generating Tags from Folders**

To generate a tags file from a folder of folders, a dictionary mapping tag to list of paths is passed to ```remo.generate_tags_from_folders()```.

In [7]:
im_list = []
types = ("*.jpg", "*jpeg", "*.png", "*.tif")
for ext in types:
    im_list.extend([os.path.basename(i) for i in glob.glob(str(path_to_images)+"/**/"+ext, recursive=True)])
im_list = random.sample(im_list, len(im_list))

tags_dict = {"train" : im_list[0:121], "test" : im_list[121:131], "valid" : im_list[131:141]}

remo.generate_tags_from_folders(tags_dict)

'train_test_valid_split.csv'

In [8]:
# The JSON file is provided in the dataset, and is then converted into a mapping dictionary.
# cat_to_index : mapping between class_index -> class_label
mapping_json_path = os.path.join(path_to_annotations, "mapping.json")
cat_to_index = dict(json.load(open(mapping_json_path)))
mapping = { value : key for (key, value) in cat_to_index.items()}

**Adding Data to Remo**

To add a dataset, you can use the ```remo.create_dataset()``` specifying the path to data and annotations. 
The class encoding is passed via a dictionary.

For a complete list of formats supported please refer the <a href="https://remo.ai/docs/annotation-formats/">documentation</a>.


In [9]:
# The annotations.csv is generated in the same path of the sub-folder
flowers = remo.create_dataset(name='train_test_valid_example', local_files=[path_to_images],
                                               annotation_task = "Image classification",
                                               class_encoding=cat_to_index)

Acquiring data - completed                                                                           
Processing data - Processing images: 5/140Processing data - Processing images: 15/140Processing data - Processing images: 25/140Processing data - Processing images: 34/140Processing data - Processing images: 43/140Processing data - Processing images: 53/140Processing data - Processing images: 63/140Processing data - Processing images: 72/140Processing data - Processing images: 81/140Processing data - Processing images: 91/140Processing data - Processing images: 100/140Processing data - Processing images: 109/140Processing data - Processing images: 120/140Processing data - Processing images: 130/140Processing data - Processing annotation files: 1/1Processing data - completed                                                                          
Data upload completed


In [10]:
annotation_id = flowers.default_annotation_set().id
flowers.add_data(paths_to_upload=["./train_test_valid_split.csv"], annotation_set_id=annotation_id)

Progress 100% - 1/1 - elapsed 0:00:00.001000 - speed: 1000.00 img / s, ETA: 0:00:00
Acquiring data - completed                                                                           
Processing data - Processing annotation files: 1/1Processing data - completed                                                                          
Data upload completed


{'session_id': '848ccb2b-a52a-4a93-ace7-c23c3ccf7a63',
 'created_at': '2020-07-30T14:30:29.045188Z',
 'finished_at': '2020-07-30T14:30:38.884136Z',
 'dataset': {'id': 52, 'name': 'train_test_valid_example'},
 'status': 'done',
 'substatus': '',
 'images': {'pending': 0,
  'total': 0,
  'successful': 0,
  'failed': 0,
  'errors': [],
  'warnings': []},
 'annotations': {'pending': 0,
  'total': 1,
  'successful': 1,
  'failed': 0,
  'errors': [],
  'warnings': []},
 'errors': [],
 'warnings': [],
 'uploaded': {'total': {'items': 1, 'size': 3225, 'human_size': '3.1 Kb'},
  'images': {'items': 0, 'size': 0, 'human_size': '0 b'},
  'annotations': {'items': 1, 'size': 3225, 'human_size': '3.1 Kb'},
  'archives': {'items': 0, 'size': 0, 'human_size': '0 b'}}}

**Visualizing the dataset**

To view your data and labels using the Remo visual interface directly in the notebook, call the ```dataset.view()``` method.

In [11]:
# Un-comment to view the other datasets.
flowers.view()

Open http://localhost:8123/datasets/52


**Dataset Statistics**

Remo alleviates the need to write extra boilerplate for accessing dataset properties. 

This can be done either using code, or via the visual interface.

In [12]:
data_stats = flowers.get_annotation_statistics()
pprint("Dataset Statistics {}".format(data_stats))

("Dataset Statistics [{'AnnotationSet ID': 64, 'AnnotationSet name': 'Image "
 "classification', 'n_images': 0, 'n_classes': 3, 'n_objects': 0, "
 "'top_3_classes': [{'name': 'Hard-leaved pocket orchid', 'count': 60}, "
 "{'name': 'Canterbury bells', 'count': 40}, {'name': 'Pink primrose', "
 "'count': 40}], 'creation_date': None, 'last_modified_date': "
 "'2020-07-30T14:30:24.329559Z'}]")


In [13]:
# Un-comment the other lines to view the other datasets.
flowers.view_annotation_stats()

Open http://localhost:8123/annotation-detail/64/insights


**Export Annotations To File**

Using the ```dataset.export_annotations_to_file()``` method, the annotations from Remo can be exported to a format of your choice.

For a complete list of formats supported please refer the <a href="https://remo.ai/docs/annotation-formats/">documentation</a>.


In [14]:
flowers.export_annotations_to_file("flowers_annotations.zip", annotation_format="csv", full_path=True, export_tags=True)
!unzip flowers_annotations.zip

Archive:  flowers_annotations.zip
  inflating: tags.csv                
  inflating: Image classification.csv  


## Feeding Data into PyTorch

A custom PyTorch ```Dataset``` object defined below is used to load data.

In order to adapt this to your dataset, the following are required:

- **Path to data:** Path to the Data Folder
- **Path to Annotations:** Path to Annotations CSV File (Format : file_name, class_name)
- **Mapping:** Python dictionary containing mapping of class name and class index (Format : {"class_name" : "class_index"})
- **transforms:** Transforms to be applied to the images before passing it to the network.

In [15]:
class FlowerDataset(Dataset):
    def __init__(self, annotations, train_test_valid_split, data_path, mapping, mode, transform=None):
        # Pandas is used to read in the csv file into a DataFrame for data loading
        self.data = pd.read_csv(annotations)
        self.train_test_valid_split = pd.read_csv(train_test_valid_split)
        self.data["tag"] = self.train_test_valid_split["tag"]
        self.data_path = data_path
        self.mapping = mapping
        self.transform = transform
        self.mode = mode
        
        if self.mode == "train":
            self.data_df = self.data[self.data["tag"] == "train"][["file_name", "classes"]].reset_index(drop=True)
        elif self.mode == "test":
            self.data_df = self.data[self.data["tag"] == "test"][["file_name", "classes"]].reset_index(drop=True)
        elif self.mode == "valid":
            self.data_df = self.data[self.data["tag"] == "valid"][["file_name", "classes"]].reset_index(drop=True)
    
    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
            labels = int(self.mapping[self.data_df.loc[idx, 'classes'].lower()])
            im_path = os.path.join(self.data_path,str(labels), self.data_df.loc[idx, 'file_name'])
            
            label_tensor = torch.as_tensor(labels-1, dtype=torch.long)
            im = Image.open(im_path)
        
            if self.transform:
                im = self.transform(im)
        
            if self.mode == "test":
                # For saving the predicitions, the file name is required
                return {"im" : im, "labels": label_tensor, "im_name" : self.data_df.loc[idx, 'file_name']}
            else:
                return {"im" : im, "labels" : label_tensor}

In [16]:
# Channel wise mean and standard deviation for normalizing according to ImageNet Statistics
means = [0.485, 0.456, 0.406]
stds = [0.229, 0.224, 0.225]


# Transforms to be applied to Train-Test-Validation
train_transforms = transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(means, stds)
    ])

val_transforms = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(means, stds)])

test_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(means, stds)
    ])

The train, test and validation datasets are instantiated and wrapped around a ```DataLoader``` method.

In [17]:
train_dataset = FlowerDataset(annotations="Image classification.csv",
                                   data_path='./flowers/',
                                   train_test_valid_split="tags.csv",
                                   transform=train_transforms,
                                   mapping=mapping, mode="train")

val_dataset = FlowerDataset(annotations="Image classification.csv",
                              data_path='./flowers',
                               train_test_valid_split="tags.csv",
                              transform=val_transforms,
                              mapping=mapping,
                               mode="valid")

test_dataset = FlowerDataset(annotations="Image classification.csv",
                              data_path='./flowers',
                              train_test_valid_split="tags.csv",
                              transform=test_transform,
                              mapping=mapping,
                              mode="test")

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=2,  shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=1, shuffle=False, num_workers=2)

data_loader = {"train" : train_loader, "valid": val_loader}
len_dict = {"train" : len(train_dataset), "valid" : len(val_dataset)}

## Training the Model

The pre-trained weights of the ```ResNet-18``` model with ImageNet are used in this tutorial.

To train the model, the following details are passed to the ```train_model()``` function

1. **Model:** The edited version of the pre-trained model.
2. **Data Loaders:** The dictionary containing our training and validation dataloaders
3. **Criterion:** The loss function used for training the network
4. **Num_epochs:** The number of epochs for which we would like to train the network.
5. **dataset_size:** an additional parameter which is used to correctly scale the loss, the method for this is specified in the DataLoader cell

In [18]:
model = models.resnet18(pretrained=True)

# Freezing the weights

for param in model.parameters():
    param.required_grad = False


# Replacing the final layer
model.fc = nn.Sequential(nn.Linear(512, 256), nn.ReLU(), nn.Dropout(p=0.5), nn.Linear(256, 102),
                         nn.LogSoftmax(dim=1))

In [19]:
def train_model(model, data_loaders, optimizer, criterion, num_epochs, dataset_size):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Checks if GPU is present

    model.to(device) # This method pushes the model to the device.
    
    # The training loop trains the model for the total number of epochs,
    # an epoch is one complete pass over the entire dataset
    for epoch in range(num_epochs):
        
        model.train() # This sets the model back to training after the validation step
        print("Epoch Number {}".format(epoch))

        training_loss = 0.0
        val_loss = 0.0
        val_acc = 0
        correct_preds = 0
        best_acc = 0
        validation = 0.0
        total = 0

        
        data_loader = tqdm.tqdm(data_loaders["train"])
        for x, data in enumerate(data_loader):
            inputs, labels = data["im"].to(device), data["labels"].to(device)
            outputs = model(inputs)
            optimizer.zero_grad()

            loss = criterion(outputs, labels)


            loss.backward()
            optimizer.step()

            training_loss += loss.item()
        
        epoch_loss = training_loss / dataset_size["train"]

        print("Training Loss : {:.5f}".format(epoch_loss))

        val_data_loader = tqdm.tqdm(data_loaders["valid"])
        
        # Validation step after every epoch
        # The gradients are not required at inference time, hence the model is set to eval mode
        with torch.no_grad():
            model.eval()
            for x, data in enumerate(val_data_loader):
                inputs, labels = data["im"].to(device), data["labels"].to(device)
                outputs = model(inputs)

                val_loss = criterion(outputs, labels)
                _, index = torch.max(outputs, 1)

                total += labels.size(0)
                correct_preds += (index == labels).sum().item()

                validation += val_loss.item()

            val_acc = 100 * (correct_preds / total)

            print("Validation Loss : {:.5f}".format(validation / dataset_size["valid"]))
            print("Validation Accuracy is: {:.2f}%".format(val_acc))
            
            # The model is saved only if current validation accuracy is higher than the previous best accuracy
            if best_acc < val_acc:
                best_acc = val_acc
                model_name = "./saved_model_{:.2f}.pt".format(best_acc)
                torch.save(model, model_name)
                
            return model_name 


In [20]:
# We use the Adam optimizer, which inherits the parameters of only the trainable layers.
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
trained_model = train_model(model=model, data_loaders=data_loader, optimizer=optimizer, num_epochs=1, dataset_size=len_dict, criterion=nn.NLLLoss())

100%|██████████| 5/5 [00:00<00:00,  7.77it/s]
Validation Loss : 0.76405
Validation Accuracy is: 88.89%


In [25]:
def test_model(dataloader, model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    test_model = torch.load(model)
    
    test_model.eval()
    tk0 = tqdm.tqdm(dataloader)
    
    total = 0
    correct_preds = 0
    pred_list = {}
    
    with torch.no_grad():
        for x, data in enumerate(tk0):
            single_im, label = data["im"].to(device), data["labels"].to(device)
            im_name = data["im_name"]
            
            pred = test_model(single_im)

            _, index = torch.max(pred, 1)

            total += label.size(0)
            correct_preds += (index == label).sum().item()
            
            pred_list[os.path.basename(im_name[0])] = (index+1).item()
            
    df = pd.DataFrame(pred_list.items(), columns=['file_name', 'class_name'])
    with open("results.csv", "w") as f:
        df.to_csv(f, index=False)
    print('Accuracy of the network on the test images: %d %%' % (100 * (correct_preds / total)))

In [26]:
test_model (test_loader, trained_model)

100%|██████████| 10/10 [00:00<00:00, 16.31it/s]Accuracy of the network on the test images: 70 %



## Visualizing Predictions

For visualizing the predicted v/s original label in Remo, the predictions are added to a CSV, which is then added as an ```AnnotationSet``` to the testing_dataset.

In [29]:
classes = list(mapping.keys())
print(classes)
#annotation_set = remo.create_annotation_set("Image Classification", name="model_predictionss", dataset_id=flowers.id, classes=classes)
#flowers.add_data(local_files=["./results.csv"], annotation_set_id=annotation_set.id, class_encoding=cat_to_index)

['fire lily', 'canterbury bells', 'bolero deep blue', 'pink primrose', 'mexican aster', 'prince of wales feathers', 'moon orchid', 'globe-flower', 'grape hyacinth', 'corn poppy', 'toad lily', 'siam tulip', 'red ginger', 'spring crocus', 'alpine sea holly', 'garden phlox', 'globe thistle', 'tiger lily', 'ball moss', 'love in the mist', 'monkshood', 'blackberry lily', 'spear thistle', 'balloon flower', 'blanket flower', 'king protea', 'oxeye daisy', 'yellow iris', 'cautleya spicata', 'carnation', 'silverbush', 'bearded iris', 'black-eyed susan', 'windflower', 'japanese anemone', 'giant white arum lily', 'great masterwort', 'sweet pea', 'tree mallow', 'trumpet creeper', 'daffodil', 'pincushion flower', 'hard-leaved pocket orchid', 'sunflower', 'osteospermum', 'tree poppy', 'desert-rose', 'bromelia', 'magnolia', 'english marigold', 'bee balm', 'stemless gentian', 'mallow', 'gaura', 'lenten rose', 'marigold', 'orange dahlia', 'buttercup', 'pelargonium', 'ruby-lipped cattleya', 'hippeastrum'

In [ ]:
flowers.view()